$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$
$\newcommand{\expec}[3]{\left\langle{#1}\left|{#2}\right|{#3}\right\rangle}$
$\newcommand{\braket}[2]{\left\langle{#1}|{#2}\right\rangle}$
$\newcommand{\op}[2]{\left|{#1}\right\rangle\left\langle{#2}\right|}$
$\newcommand{\X}{\mathrm{X}}$
$\newcommand{\Y}{\mathrm{Y}}$
$\newcommand{\Z}{\mathrm{Z}}$
$\newcommand{\H}{\mathrm{H}}$
$\newcommand{\CX}{\mathrm{CNOT}}$
$\newcommand{\RX}{\mathrm{RX}}$
$\newcommand{\RY}{\mathrm{RY}}$
$\newcommand{\RZ}{\mathrm{RZ}}$

# Task 1: Basis Encoding and Amplitude Amplification

Let's see an example of how basis encoding can be applied to an example problem. We'll also introduce a generalisation of Grover's algorithm to the case where there are multiple solutions to a problem.

Imagine you are invited into the lab of your chemist friend to see a new experiment they've developed. They lay out a few chemical samples on the desk: water, sodium, iron, oxygen, fluorine, hydrogen, uranium and lead. Suddenly your friend is called away to an explosive emergency in the undergraduate teaching labs. 

They throw on their protective goggles and ask you to put away the samples into two boxes on the shelf. As they run frantically out the door your chemist friend turns to you and says, "check the warning labels and no matter what - don't mix the wrong samples together in one box".

The labels on the samples give you the following information:

- **Sodium**: do not store with *water*
- **Iron**: do not store with *water*
- **Fluorine**: do not store with *sodium* or *hydrogen*
- **Hydrogen**: do not store with *oxygen*
- **Uranium**: must be stored with *water* or *lead*
- **Lead**: do nost store with *sodium*

![image](https://drive.google.com/uc?export=view&id=1x2j8Og-ObHIvvu1tUhCnwgvT6EABHfOw)

As a physicist, you decide that checking the combinations by hand is too time-consuming, and so you decide instead to design a quantum algorithm that can find a combination that satisfies these criteria, and solve the problem for you.

To solve this problem we will apply Grover's algorithm, recalling the algorithm:

1. First encode all $N$ possible solutions in the computational basis states, and create an equal superposition $\ket{s}$. 
2. Apply the oracle $U_w$ to invert the phase of all $M$ correct solutions
3. Apply a reflection $U_s$ about the starting vector $\ket{s}$
4. Repeat application of $U_sU_w$ approximately $\sqrt{N/M}$ times in order to amplify the correct solutions and then perform a measurement

### __Encoding the Problem__

The encoding scheme is straightforward: we create a register of 5 qubits, each representing one of the samples. The qubit in the 1 state represents the sample being placed in Box A, and the qubit in the 0 state represents the sample being placed in Box B. 

For example, a trial solution with water, sodium, oxygen and uranium in box A, and iron, fluorine, hydrogen and lead in box B would be represented by the 8-qubit computational basis state:

$$
\ket{11010010}
$$

The hard part is designing the oracle that inverts only the correct solutions.

### __Quantum Oracle__

We construct the oracle by formulating the problem in terms of a set of logical expressions that evaluate to true if the proposed solution is correct, and to false if it is incorrect.

The result from evaluating each expression is stored in an ancillary qubit, with the state $\ket{1}$ if the expression is true, or the state $\ket{0}$ if the expression is false. The oracle then determines whether the input is a solution by checking that each ancillary qubit is in the state $\ket{1}$ (i.e true) and storing the final result in an additional ancillary qubit.

The last step of creating the oracle is to apply a technique known as *phase kickback*, which applies the eigenvalue of a measurement to the control qubits. In this case, if the final ancillary qubit evaluates to $\ket{1}$ then the result of a $Z$ measurement is $-1$ which inverts the phase of the control qubits.

Below is an example of an oracle that checks whether the first two conditions are satisfied for box A.


In [ ]:
from qat.lang.AQASM import QRoutine, CCNOT, Program, CNOT, H, Z
from qat.qpus import get_default_qpu

def oracle():
    rout = QRoutine()
    qbits = rout.new_wires(3)
    qanc = rout.new_wires(3) #ancillary qubits
    with rout.compute():
      CNOT(qbits[0], qanc[0])
      CNOT(qbits[1], qanc[0])
      CNOT(qbits[0], qanc[1])
      CNOT(qbits[2], qanc[1])
      CCNOT(qanc[0], qanc[1], qanc[2])
    Z(qanc[2])
    rout.uncompute() #uncompute provides the phase kickback
    return rout

Now let's implement this as part of the amplitude amplification algorithm.

In [ ]:
prog = Program()
qbits = prog.qalloc(6)

for qbit in qbits[:3]: #create the equal superposition
  H(qbit)

prog.apply(oracle(), qbits) #apply the oracle

#check how application of the oracle has changed the state of the qubit register
circ = prog.to_circ()
result = get_default_qpu().submit(circ.to_job())
for s, sample in enumerate(result):
    print(f"{sample.amplitude:<24} {sample.state}")

The two solution states are those with negative phases, i.e

\begin{align}
  \ket{011} &\implies \text{water in box A, sodium and iron in box B}\\
  \ket{100} &\implies \text{water in box B, sodium and iron in box A}
\end{align}

Lets look at the circuit we've just implemented

In [ ]:
%qatdisplay circ

In the example above we used two CNOT gates to create each XOR condition. Thankfully myQLM provides tools that mean we don't have to determine which combinations of gates evaluate a boolean expression, which is a formidable challenge for large problems with lots of conditions.

With myQLM we can declare the qubit register as a boolean array and create these expressions using the native Python bitwise operators:

$$
\begin{matrix}
\text{Operator} & \text{Name} \\
\& & \text{AND} \\
| & \text{OR} \\
\hat{} & \text{XOR} \\
\tilde{} & \text{NOT}
\end{matrix}
$$

Doing this also allows us to apply the phase kickback using a simple `.phase()` method and do away with the ancillary qubits, which are assigned automatically.

In [ ]:
from qat.lang.AQASM.qbool import QBoolArray

def oracle():
    rout = QRoutine()
    qbools = rout.new_wires(3, QBoolArray) #tell myQLM to treat register as boolean values
    expr1 = qbools[0] ^ qbools[1] #water xor sodium
    expr2 = qbools[0] ^ qbools[2] #water xor iron
    expr = expr1 & expr2 #check each expression
    expr.phase()
    return rout

The second step in the amplitude amplification is the so-called diffusion operator, which reflects the qubit state through the original superposition vector

$$
U_s = 2\op{s}{s} - 1
$$

This gate is straightforward to construct in matrix form using the above definition, and myQLM allows us to construct a gate from a function that returns a matrix using the `AbstractGate` class. 

In [ ]:
from qat.lang.AQASM import AbstractGate
import numpy as np

def s_flip(nqbits):
    dimen = 2**nqbits
    s_vec = np.ones(dimen)/np.sqrt(dimen)
    s_mat = np.outer(s_vec, s_vec)
    s_flip = 2*s_mat - np.identity(dimen)
    return s_flip

Sflip = AbstractGate("S_flip", [int], matrix_generator=s_flip)

Putting everything together we can apply the complete amplitude amplification algorithm. In this case the database is very small and a single application of the oracle and diffusion operators, $U_sU_w$, is able to rotate the state vector exactly onto the superposition of correct solutions $\ket{w}$.

**Construct a quantum program below that applies a single iteration of Grover's algorithm, using the Oracle function and the diffusion function defined above**

In [ ]:
################################
### Create your program here ###
################################

prog = Program()
qbools = prog.qalloc(3, QBoolArray)

for qbit in qbools:
    H(qbit)

for i in range(1): #a single repetition of the oracle + diffusion is sufficient
  prog.apply(oracle(), qbools)
  prog.apply(Sflip(3), qbools)

circ = prog.to_circ()
job = circ.to_job(nbshots=0)

from qat.qpus import get_default_qpu

result = get_default_qpu().submit(job)
for s, sample in enumerate(result):
  print(f"{s}: {sample.state} {sample.amplitude}")

We mentioned above that the oracle and diffusion operators need to be repeated approximately $\sqrt{N/M}$ in order to measure a correct solution with high probability. Often the number of correct solutions $M$ is not known in advance, and hence the number of repetitions required is not obvious. 

In this case we can repeatedly run the algorithm with an increasing number of repetitions $1, 2, 3, 4, ...$ until a correct solution is found. The total number of iterations taken is still $O(\sqrt{N/M})$ and a quadratic speed-up over the best classical case is still achieved, up to a constant factor.

**Try adapting the oracle function given previously for the partial problem and extend the circuit to 8 qubits in order to solve the complete boxing problem with all the conditions given at the top of the section.**


In [ ]:
############################################################
### Create your oracle function and quantum program here ###
############################################################

def oracle():
    rout = QRoutine()
    qbools = rout.new_wires(8, QBoolArray)
    expr1 = qbools[0] ^ qbools[1] #water or sodium
    expr2 = qbools[0] ^ qbools[2] #water or iron
    expr3 = qbools[1] ^ qbools[4] #fluorine or sodium
    expr4 = qbools[4] ^ qbools[5] #fluorine or hydrogen
    expr5 = qbools[5] ^ qbools[3] #hydrogen or oxygen
    expr6 = qbools[6] & (qbools[7] | qbools[0]) #uranium and lead or water
    expr7 = qbools[7] ^ qbools[4] #lead or fluorine
    expr = expr1 & expr2 & expr3 & expr4 & expr5 & expr6 & expr7
    expr.phase()
    return rout

reps=11
prog = Program()
qbools = prog.qalloc(8, QBoolArray)

for qbit in qbools:
    H(qbit)
for i in range(reps):
    prog.apply(oracle(), qbools)
    prog.apply(Sflip(8), qbools)

circ = prog.to_circ()
#circ.display()
job = circ.to_job(nbshots=0)

from qat.qpus import get_default_qpu

result = get_default_qpu().submit(job)
for s, sample in enumerate(result):
    if abs(sample.amplitude) > 0.5:
        print(f"The state: {sample.state}\n  has amplitude {sample.amplitude}")

# Task 2: Angle Encoding and Quantum Neural Networks

Machine learning problems can generally be cast in terms of vector-matrix products and a series of linear algebraic operations. Quantum circuits act as very efficient simulators for linear algebra and can perform linear algebraic calculations with far fewer operations.

In this example we'll create a QNN that learns to classify a dataset describing attributes of stars into pulsars and non-pulsars. The data in question is the HTRU2 dataset, which contains almost 18,000 entries. Each entry in the database has a 'label' with the value 0 or 1 depending on whether the entry is a pulsar or not, and is described by 8 features:

1. Mean of the integrated profile.
2. Standard deviation of the integrated profile.
3. Excess kurtosis of the integrated profile.
4. Skewness of the integrated profile.
5. Mean of the DM-SNR curve.
6. Standard deviation of the DM-SNR curve.
7. Excess kurtosis of the DM-SNR curve.
8. Skewness of the DM-SNR curve.

__Outline of the QNN__

The basic format of a quantum neural network interlaces feature encoding layers with trainable rotations. 

In the figure below we encode $n$ features via $R_z$ rotations. Each feature encoding is followed by gate(s) parameterised by a vector of trainable weights $\vec{w}^{(i)}$. The QNN repeats the entire feature encoding block $N$ times to improve the expressibility of the model.

![image](https://drive.google.com/uc?export=view&id=1_J22wva1AcJWdAvlu4cFf0UZIcODHN1x)

The aim is to construct a variational quantum circuit that rotates the state vector towards the $\ket{1}$ state if the features correspond to a pulsar, and toward the $\ket{0}$ state if the features do not correspond to a pulsar.

To make a prediction we run the circuit several times and measure the qubit to obtain the probability of measuring the state $\ket{1}$. We interpret this result as the probability of a data point corresponding to a pulsar.

$$
p_1 = \frac{\langle Z \rangle + 1}{2} = \left\langle \frac{Z + 1}{2} \right\rangle
$$

__Optimizing the Circuit__

The circuit 'learns' to correctly identify pulsars by optimizing the parameters in the trainable rotation layers. This optimization is performed via classical methods, giving rise to a quantum-classical hybrid algorithm.

First we pass data points with known labels through the circuit and compute a loss function on the output. The value of the loss function is larger the further away from the correct label the prediction is, and zero for a correct prediction.

Then we calculate the gradient of the loss function with respect to each of the trainable weights and apply classical gradient descent methods to update the trainable weights gradually optimize the circuit to reduce the loss.

## The Variational Quantum Circuit

**Create a function that creates a QNN circuit like the one shown in the figure above with a single layer (N=1). The function should accept two arguments: a vector of weights to parameterise the trainable layers, and a vector of 8 features describing an entry in the HTRU2 database, and should return the expectation value $\langle P_1 \rangle$.** 

- Start by instantiating a program with a single qubit register and preparing the qubit in an equal superposition using a Hadamard gate.
- Next add an initial 'zeroth' trainable rotation layer with a parameterised gate of your choice.
- Then add feature encoding blocks consisting of $R_z$ angle encoding rotations followed by trainable rotations in order to encode each of the 8 features.
- Convert the program to a job with an observable measurement corresponding to the probability of measuring $\ket{1}$ and return the expectation value

In [ ]:
from qat.lang.AQASM import RZ, RX, RY
from qat.core import Observable

def init_weights(nlayers=1, ntrainablegates=1):
    return np.random.uniform(low=-1., high=1., size=((nlayers*8+1)*ntrainablegates))

def qnn_circuit(weights:np.ndarray, features:np.ndarray):
    prog = Program()
    qbits = prog.qalloc(1)
    H(qbits[0])
    RX(float(weights[0]))(qbits[0])
    RY(float(weights[1]))(qbits[0])
    for l in range(2):
        for f in range(8):
            ind = (l*n_features + f + 1)*n_trainables
            RZ(features[f])(qbits[0])
            RX(float(weights[ind]))(qbits[0])
            RY(float(weights[ind+1]))(qbits[0])
    job = prog.to_circ().to_job(observable=Observable(1, matrix=np.array([[0, 0],[0, 1]])))
    expec = get_default_qpu().submit(job).value
    return expec

Below we initialise a random set of initial weights and pass a set of features to the `qnn_circuit` function to check it executes a valid circuit.

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

def fetch_data_random_seed(n_samples, seed=0):
    """Helper function for pre-processing the data"""

    dataset = pd.read_csv('HTRU2.csv')
    
    # Import non-pulsar data points
    data0 = dataset[dataset[dataset.columns[8]] == 0]
    data0 = data0.sample(n=n_samples, random_state=seed)
    X0 = data0[data0.columns[0:8]].values
    Y0 = data0[data0.columns[8]].values

    # Import pulsar data points
    data1 = dataset[dataset[dataset.columns[8]] == 1]
    data1 = data1.sample(n=n_samples, random_state=seed)
    X1 = data1[data1.columns[0:8]].values
    Y1 = data1[data1.columns[8]].values

    X = np.append(X0, X1, axis=0)
    Y = np.append(Y0, Y1, axis=0)
    # Normalise the data
    min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, np.pi))
    X = min_max_scaler.fit_transform(X)

    # Separate the test and training datasets
    train_X, validation_X, train_Y, validation_Y = train_test_split(X, Y, test_size=0.5, random_state=seed)

    return train_X, validation_X, train_Y, validation_Y

# Sample a single instance of the dataset
train_X, validate_X, train_Y, validate_Y = fetch_data_random_seed(1)

np.random.seed(0)

print("Test expectation value: ", qnn_circuit(init_weights(), train_X[0]))

## Classical Optimization

Next we need to create the classical part of the algorithm for training the quantum model. For this we need:

1. A loss function $\mathcal{L}(p_1, y)$ that compares the prediction $p_1$ to the data point's label, $y$
2. The gradient of the loss function w.r.t each of the weights 
$$
\frac{\partial \mathcal{L}}{\partial w_i} = \frac{\partial \mathcal{L}}{\partial p_1} \frac{\partial p_1}{\partial w_i} 
$$
3. A gradient descent method to update the weights and optimize the circuit

__Loss Function__

We'll use a so-called 'cross entropy' loss function:

$$
\mathcal{L}(p_1, y) = -y \log(p_1) - (1-y) \log(1 - p_1)
$$

which has a simple analytical gradient:

$$
\frac{\partial \mathcal{L}}{\partial p_1} = \frac{1 - y}{1 - p_1} - \frac{y}{p_1}
$$

The gradient of the expectation value w.r.t the weights can be obtained by the [parameter-shift method](https://arxiv.org/pdf/2107.12390.pdf). 
This is a common and highly efficient method for obtaining the gradient of a quantum circuit with respect to each of its parameters, which expresses the gradient of an expectation value analytically in terms of a $\pi/2$ shift in the rotation angle:

$$
\frac{\partial p_1(w_i)}{\partial w_i} = p_1(w_i + \pi/2) - p_1(w_i - \pi/2)
$$

**Now create a function that accepts a set of weights and features as well as the class label for that set of features, and returns the loss and a vector of gradients with the same shape as the input gradient of weights.**

In [ ]:
## Solution divided into two functions for clarity

def gradient(weights, features):
    """Vector gradient with respect to all weights in a model"""
    gradients = np.zeros_like(weights)
    for index in range(len(weights)):
        shift = np.zeros_like(weights)
        shift[index] = np.pi/2
        gradient = .5 * (
            qnn_circuit(weights + shift, features) 
            - qnn_circuit(weights - shift, features)
        )
        gradients[index] = gradient
    return gradients

def loss_and_grad(weights:np.ndarray, features:np.ndarray, label:int):
    """Calculate loss and gradient with respect to a set of weights for a given sample"""
    expec = qnn_circuit(weights, features)
    expec_grads = gradient(weights, features)

    loss = -label*np.log(expec) - (1-label)*np.log(1-expec)
    loss_grad = (1-label)/(1-expec) - label/expec

    grads = loss_grad * expec_grads
    return loss, grads

__The Optimizer__

If you have experience with machine learning methods, the final step will be very familiar to you. There are many gradient descent methods, ranging from the simple Newton's gradient descent method to more advanced ones commonly used in machine learning applications such as the AdaGrad, Stochastic Gradient Descent, and Adam.

These optimizers have been used with a high degree of success throughout classical machine learning applications, so rather than re-inventing the wheel, we'll borrow them directly from the `optax` library. An 'optimizer' in this context is essentially an algorithm for deciding how much to update the weights based on the value of the gradient w.r.t. that weight.

We'll proceed with an optimizer known as an [Adam optimizer](https://arxiv.org/abs/1412.6980), which implements a type of batch gradient descent - where the loss and gradients are averaged over a batch of training samples to counteract the stochasticity of the dataset. The optimizer updates the weights according to an internal algorithm and a set of hyperparameters

First we create a function that iterates over all the samples in a batch and calculates the average loss and gradients.

In [ ]:
def step(weights, batch):
    """Perform a step of the optimizer by applying gradients across a batch of samples"""
    batch_loss = 0
    batch_grad = np.zeros_like(weights)
    # Calculate average loss and gradient over data samples in the batch
    for (features, label) in batch:
        loss, grad = loss_and_grad(weights, features, label)
        batch_loss += loss/len(batch)
        batch_grad += grad/len(batch)
    return batch_loss, batch_grad

Now we can initialise an optimizer, and iterate over a fixed number of epochs, applying updates to the weights after each epoch based on the average gradients.

In [ ]:
from optax import adam, apply_updates

def train(weights, data, num_epochs):
    batch = list(zip(*data))
    optimizer = adam(learning_rate=1e-1)
    opt_state = optimizer.init(weights)
    for i in range(num_epochs):
        batch_loss, batch_grad = step(weights, batch)
        # Update the array of weights using the Adam optimizer
        updates, opt_state = optimizer.update(batch_grad, opt_state, weights)
        weights = apply_updates(weights, updates)
        yield weights, batch_loss

Finally we want to see how accurately the model classifies the data by comparing to the assigned label

In [ ]:
def accuracy(weights, data):
    """Compute the accuracy of the classification for a given set of weights and validation data"""
    errors = 0
    for x, y in zip(data[0], data[1]):
        expec = qnn_circuit(weights, x)
        errors += abs(y - round(expec))
    acc = 1 - errors/len(data[0])
    return acc

def train_validate(train_data, weights, validate_data, num_epochs=150):
    for i, (weights, loss) in enumerate(train(weights, train_data, num_epochs)):
        acc = accuracy(weights, validate_data)
        print(f"Epoch {i+1}: Loss: {loss:5.3f} Accuracy: {acc*100}%")
    return

Try training the model you've created, you can also try adapting your `qnn_circuit` by trying different trainable gates, or adding multiple trainable gates after each feature encoding. Try increasing the number of layers and see how the accuracy improves.

In [ ]:
# Fetch a random sample of the HTRU2 data
train_X, validate_X, train_Y, validate_Y = fetch_data_random_seed(100, 0)

# Train and validate the model using the data sampled
train_validate((train_X, train_Y), init_weights(), (validate_X, validate_Y))